In [17]:
import requests

def convert_coordinates_to_address(lat, lng):
    """
    입력받은 위도, 경도를 도로명, 지번 주소로 변환
    """
    y, x = str(lat), str(lng)
    url = f'https://dapi.kakao.com/v2/local/geo/coord2address.json?x={x}&y={y}'
    headers = {'Authorization': 'KakaoAK 4c8046d5177ecebf6bf6fdf0154cc114'}
 
    r = requests.get(url, headers=headers)
    
    # 상태 코드가 200일 때만 처리
    if r.status_code == 200:
        json_data = r.json()
        documents = json_data.get("documents")
        if not documents:
            # 응답에 문서가 없으면 None 반환
            return None
        
        doc = documents[0]
        
        # road_address와 address가 있는지 확인하고, 없으면 None 할당
        bunji_address_data = doc.get("address")
        
        bunji_address = bunji_address_data.get('address_name') if bunji_address_data else None
    else:
        return None
    
    return bunji_address

# TEST
result = convert_coordinates_to_address('37.5020838257097',	'126.90471572236515')
print(result)


서울 영등포구 신길동 400-1


In [ ]:
import geopandas as gpd


,latitude,longitude,score,SIG_KOR_NM,geometry
0,37.466698724120896,126.89050883155603,0.602538773953227,금천구,POINT (126.89051 37.4667)
1,37.5020838257097,126.90471572236515,0.5787335334598886,영등포구,POINT (126.90472 37.50208)
2,37.529939903626136,127.07459046142202,0.5745836940342952,광진구,POINT (127.07459 37.52994)
3,37.47247653659869,127.04498066380205,0.5591038360167819,서초구,POINT (127.04498 37.47248)
4,37.491489545131415,127.08167492828633,0.5461847875097152,강남구,POINT (127.08167 37.49149)


In [18]:
import geopandas as gpd
import requests
import time

def convert_coordinates_to_bunji_address(lat, lng):
    """
    입력받은 위도, 경도를 지번 주소로 변환하여 반환합니다.
    """
    y, x = str(lat), str(lng)
    url = f'https://dapi.kakao.com/v2/local/geo/coord2address.json?x={x}&y={y}'
    headers = {'Authorization': 'KakaoAK 4c8046d5177ecebf6bf6fdf0154cc114'}  # 본인의 API 키로 교체하세요.
 
    r = requests.get(url, headers=headers)
    
    if r.status_code == 200:
        json_data = r.json()
        documents = json_data.get("documents")
        if not documents:
            return None
        doc = documents[0]
        # 지번 주소만 추출 (API 응답에서 "address" 키에 지번 정보가 있음)
        address_data = doc.get("address")
        bunji_address = address_data.get('address_name') if address_data else None
    else:
        return None
    
    return bunji_address

# GeoJSON 파일 읽어오기 (geopandas 사용)
gdf = gpd.read_file("rc_trash_bins.geojson")

# 각 행의 latitude, longitude로 지번 주소를 조회해 새로운 칼럼에 저장하기
def get_bunji_address(row):
    address = convert_coordinates_to_bunji_address(row["latitude"], row["longitude"])
    # API 호출 과부하 방지를 위해 잠시 대기 (필요시 조정)
    time.sleep(0.1)
    return address

gdf["bunji_address"] = gdf.apply(get_bunji_address, axis=1)

# 결과 확인
print(gdf.head())

# 변경된 GeoDataFrame을 새 파일로 저장 (선택 사항)
gdf.to_file("rc_trash_bins_with_bunji_address.geojson", driver="GeoJSON")


             latitude           longitude               score SIG_KOR_NM  \
0  37.466698724120896  126.89050883155603   0.602538773953227        금천구   
1    37.5020838257097  126.90471572236515  0.5787335334598886       영등포구   
2  37.529939903626136  127.07459046142202  0.5745836940342952        광진구   
3   37.47247653659869  127.04498066380205  0.5591038360167819        서초구   
4  37.491489545131415  127.08167492828633  0.5461847875097152        강남구   

                     geometry      bunji_address  
0   POINT (126.89051 37.4667)  서울 금천구 독산동 1082-4  
1  POINT (126.90472 37.50208)  서울 영등포구 신길동 400-1  
2  POINT (127.07459 37.52994)     서울 광진구 자양동 738  
3  POINT (127.04498 37.47248)  서울 서초구 양재동 291-25  
4  POINT (127.08167 37.49149)   서울 강남구 일원동 706-5  


In [19]:
temp = gpd.read_file('rc_trash_bins_with_bunji_address.geojson')
temp.head()

,latitude,longitude,score,SIG_KOR_NM,bunji_address,geometry
0,37.466698724120896,126.89050883155603,0.602538773953227,금천구,서울 금천구 독산동 1082-4,POINT (126.89051 37.4667)
1,37.5020838257097,126.90471572236515,0.5787335334598886,영등포구,서울 영등포구 신길동 400-1,POINT (126.90472 37.50208)
2,37.529939903626136,127.07459046142202,0.5745836940342952,광진구,서울 광진구 자양동 738,POINT (127.07459 37.52994)
3,37.47247653659869,127.04498066380205,0.5591038360167819,서초구,서울 서초구 양재동 291-25,POINT (127.04498 37.47248)
4,37.491489545131415,127.08167492828633,0.5461847875097152,강남구,서울 강남구 일원동 706-5,POINT (127.08167 37.49149)


In [21]:
import geopandas as gpd
import requests
import time
from tqdm import tqdm

# pandas에 tqdm progress_apply 확장 활성화
tqdm.pandas()

def convert_jibeon_to_road(jibeon_address, kakao_api_key):
    """
    지번 주소를 입력받아 Kakao API를 통해 도로명 주소로 변환합니다.
    :param jibeon_address: 변환할 지번 주소 (예: "서울특별시 강남구 역삼동 123-45")
    :param kakao_api_key: Kakao REST API 키
    :return: 도로명 주소 문자열 또는 오류 메시지
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {
        "Authorization": f"KakaoAK 4c8046d5177ecebf6bf6fdf0154cc114"
    }
    params = {
        "query": jibeon_address
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return f"API 요청 실패: {response.status_code}"
    data = response.json()
    # 검색 결과가 없는 경우
    if not data.get("documents"):
        return "검색 결과가 없습니다."
    # 첫 번째 결과에서 도로명 주소 정보가 있는지 확인
    for document in data["documents"]:
        road_info = document.get("road_address")
        if road_info and road_info.get("address_name"):
            return road_info["address_name"]
    return "도로명 주소 정보가 없습니다."

if __name__ == "__main__":
    # 자신의 Kakao API 키로 변경하세요.
    kakao_api_key = "4c8046d5177ecebf6bf6fdf0154cc114"
    
    # GeoJSON 파일 읽어오기
    gdf = gpd.read_file("rc_trash_bins_with_bunji_address.geojson")
    
    # 각 행의 bunji_address 값을 도로명 주소로 변환하여 새 열에 저장하는 함수
    def get_road_address(bunji_addr):
        # bunji_addr가 비어있으면 None 반환
        if not bunji_addr:
            return None
        road_addr = convert_jibeon_to_road(bunji_addr, kakao_api_key)
        # API 호출 간격 조절 (필요에 따라 지연시간 조정)
        time.sleep(0.1)
        return road_addr
    
    # progress_apply()를 사용하여 진행 상황 표시와 함께 새 열 추가
    gdf["road_address"] = gdf["bunji_address"].progress_apply(get_road_address)
    
    # 결과 일부 출력하여 확인
    print(gdf[["bunji_address", "road_address"]].head())
    
    # 변경된 GeoDataFrame을 새 GeoJSON 파일로 저장 (필요한 경우)
    gdf.to_file("rc_trash_bins_with_road_address_updated.geojson", driver="GeoJSON")


100%|██████████| 1500/1500 [04:34<00:00,  5.46it/s]


       bunji_address      road_address
0  서울 금천구 독산동 1082-4  도로명 주소 정보가 없습니다.
1  서울 영등포구 신길동 400-1  도로명 주소 정보가 없습니다.
2     서울 광진구 자양동 738  서울 광진구 자양번영로 8-2
3  서울 서초구 양재동 291-25  서울 서초구 동산로13길 16
4   서울 강남구 일원동 706-5  도로명 주소 정보가 없습니다.


In [22]:
temp = gpd.read_file("rc_trash_bins_with_road_address_updated.geojson")
temp.head()


,latitude,longitude,score,SIG_KOR_NM,bunji_address,road_address,geometry
0,37.466698724120896,126.89050883155603,0.602538773953227,금천구,서울 금천구 독산동 1082-4,도로명 주소 정보가 없습니다.,POINT (126.89051 37.4667)
1,37.5020838257097,126.90471572236515,0.5787335334598886,영등포구,서울 영등포구 신길동 400-1,도로명 주소 정보가 없습니다.,POINT (126.90472 37.50208)
2,37.529939903626136,127.07459046142202,0.5745836940342952,광진구,서울 광진구 자양동 738,서울 광진구 자양번영로 8-2,POINT (127.07459 37.52994)
3,37.47247653659869,127.04498066380205,0.5591038360167819,서초구,서울 서초구 양재동 291-25,서울 서초구 동산로13길 16,POINT (127.04498 37.47248)
4,37.491489545131415,127.08167492828633,0.5461847875097152,강남구,서울 강남구 일원동 706-5,도로명 주소 정보가 없습니다.,POINT (127.08167 37.49149)


In [23]:
# 파일 불러오기
input_filename = "rc_trash_bins_with_road_address_updated.geojson"
output_filename = "rc_trash_bins_with_final_address.geojson"
gdf = gpd.read_file(input_filename)

# 새로운 '주소' 칼럼 추가 함수
def select_address(row):
    # 도로명 주소가 존재하며 "도로명 주소 정보가 없습니다."가 아니라면 도로명 주소 사용
    if row.get("road_address") and row["road_address"] != "도로명 주소 정보가 없습니다.":
        return row["road_address"]
    # 그렇지 않으면 지번 주소 사용
    return row.get("bunji_address", None)

# apply를 사용해 각 행마다 '주소' 값을 결정
gdf["주소"] = gdf.apply(select_address, axis=1)

# 결과 확인 (필요시)
display(gdf[["bunji_address", "road_address", "주소"]].head())

# 변경된 GeoDataFrame을 새로운 GeoJSON 파일로 저장
gdf.to_file(output_filename, driver="GeoJSON")

,bunji_address,road_address,주소
0,서울 금천구 독산동 1082-4,도로명 주소 정보가 없습니다.,서울 금천구 독산동 1082-4
1,서울 영등포구 신길동 400-1,도로명 주소 정보가 없습니다.,서울 영등포구 신길동 400-1
2,서울 광진구 자양동 738,서울 광진구 자양번영로 8-2,서울 광진구 자양번영로 8-2
3,서울 서초구 양재동 291-25,서울 서초구 동산로13길 16,서울 서초구 동산로13길 16
4,서울 강남구 일원동 706-5,도로명 주소 정보가 없습니다.,서울 강남구 일원동 706-5
